Make that we are running the correct version of TensorFlow first

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [2]:
import sys

assert sys.version_info >= (3, 6) # Python ≥3.6 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

# Train and use lenet5 for character predication

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')

from tensorflow.data import Dataset

In [4]:
# Import traps: http://python-notes.curiousefficiency.org/en/latest/python_concepts/import_traps.html
from recognizer.datasets import EmnistDataset
from recognizer.networks import lenet5

Hyperparameters

In [26]:
batch_size = 128
epochs = 1

Setup dataset

In [27]:
emnist = EmnistDataset()

train_dataset = emnist.train_dataset.shuffle(1024).batch(batch_size)
test_dataset = emnist.test_dataset.batch(batch_size)

Download path: /Users/mostafagazar/Work/notebooks/line-reader/data/cache/datasets/matlab.zip
Processing data...
Balancing train dataset...
Target max number of images per class: 11256
Dataset ready


In [28]:
train_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None, 1)), types: (tf.uint8, tf.uint8)>

In [35]:
(x_train, y_train), = train_dataset.take(1)
input_shape = x_train[0].shape
print(f"x shape: {x_train.shape}, model input shape: {input_shape}")

x shape: (128, 28, 28, 1), model input shape: (28, 28, 1)


Setup Network

In [30]:
model = lenet5(input_shape=input_shape, number_of_classes=emnist.number_of_classes)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 6)         60        
_________________________________________________________________
average_pooling2d_6 (Average (None, 13, 13, 6)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 11, 16)        880       
_________________________________________________________________
average_pooling2d_7 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_10 (Dense)             (None, 84)               

In [31]:
(x_test, y_test), = test_dataset.take(1)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x=x_train, y=y_train, epochs=5)
model.evaluate(x_test, y_test)

# for epoch in range(epochs):
#     for batch, (images, labels) in enumerate(train_dataset, 1):
#         pass

Epoch 1/5
128/128 [==============================] - 0s 1ms/sample - loss: 0.0000e+00 - accuracy: 0.1094
Epoch 2/5
128/128 [==============================] - 0s 207us/sample - loss: 0.0000e+00 - accuracy: 0.0234
Epoch 3/5
128/128 [==============================] - 0s 212us/sample - loss: 0.0000e+00 - accuracy: 0.0156
Epoch 4/5
128/128 [==============================] - 0s 179us/sample - loss: 0.0000e+00 - accuracy: 0.0312
Epoch 5/5
128/128 [==============================] - 0s 504us/sample - loss: 6638.2838 - accuracy: 0.0938


[6638.2838134765625, 0.09375]